'''
OBJECTIVES:
1. Build WRS system
2. Build Structural BMP Solution evaluator
3. Identify minimum BMP solution front for:
   individual facilities
   facilities w/in departments
   facilities w/in city
   
PYTHON VERSION: 3.6.3  
SQLALCHEMY VERSION: 1.1.13

'''

In [1]:
import winsound
import pandas as pd
'''
Define basic SQLAlchemy items:
    declarative base object
    connection object
    session object
    DB tables
'''
#SQLAlchemy library items:
from sqlalchemy import create_engine
from sqlalchemy import Column, Integer, String
from sqlalchemy import update, insert
from sqlalchemy import and_ #used in query.filter() to joing multiple where clauses
from sqlalchemy import ForeignKey
from sqlalchemy.orm import relationship #http://docs.sqlalchemy.org/en/latest/orm/basic_relationships.html#relationship-patterns
from sqlalchemy import inspect

from SQLA_Base import Base #module containing declarative_base
from SQLA_conn_man import session, engine #module handling db and connection creation 

#Table definitions as SQLA classes:
from SQLA_DB_base_bmp_feasibility_test_results import Base_BMP_Feasibility_Test_Results as BBFTR
from SQLA_DB_base_bmp_feasibility_test_definitions import Base_BMP_Feasibility_Test_Definitions as BBFTD
from SQLA_DB_base_bmps import Base_BMPs
from SQLA_DB_combo_bmps import Combo_BMPs
from SQLA_DB_combo_bmp_feasibility_test_results import Combo_BMP_Feasibility_Test_Results as CBFTR
from SQLA_DB_expressions import Expressions
from SQLA_DB_facility_chars import Facility_Chars
from SQLA_DB_facility_monthly_rain import Facility_Monthly_Rain
from SQLA_DB_facility_risks import Facility_Risks
from SQLA_DB_facility_type_has_nel import Facility_Type_Has_NEL
from SQLA_DB_facility_types import Facility_Types
from SQLA_DB_feasibility_test_questions import Feasibility_Test_Questions as FTQ
from SQLA_DB_nel_sample_classes import NEL_Sample_Classes
from SQLA_DB_existing_pollutant_concentrations import Existing_Pollutant_Concentrations as ExPollConcs
from SQLA_DB_pollutant_removal_rates import Pollutant_Removal_Rates as PRR
from SQLA_DB_wrs_pollutant_risks import WRS_Pollutant_Risks
Base.metadata.create_all(engine, checkfirst=True) #create SQLA classes

'''
Dictionary of "SQLAlchemy where clause lambda functions" that importCSV uses to test record uniqueness.
used as the where clause in sqlalchemy queries, updates and deletes 
Form:
    {TableName:Lambda Function, TableName:Lambda Function, ...}
    
    TableName is the table name we want to define uniqueness test for
    Lambda Function is a SQLAlchemy query used to test record uniqueness. The function can take on any form 
        but must be made to evaluate the CSV row passed as a dictionary (CSVRowDict in this explanation):
        CSVRowDict: {FieldName:CSVColValue, DBTableFieldName:CSVColValue...} 
            Where: DBTableFieldName is the name of the field associated with the value at CSVColValue on the current row
                   CSVColValue: a value in the CSV's current row+column corresponding to the DBTableFieldName 
        *this assumes that field names are unique across table. if not, then method fails (maybe need to extend method?)
    FALSE: indicates that db table doesn't impose uniqueness on its records (other than its record id being unique)
        
e.g.: lambda myRowVal: Base.metadata.tables['people'].c['name'] == CSVRowDict['name']
        using lambda function in query will search for CSVRowDict's value for 'name' in the table people, field name 
if table has no record uniqueness requirement, then enter: TableName:False
'''
unqTests = {
    'facility_chars': lambda CSVRowDict: Base.metadata.tables['facility_chars'].c['Fac_Name'] == CSVRowDict['Fac_Name'],
    'facility_monthly_rain': False, #DB schema does not impose uniqueness on records in this table
    'facility_type_has_nel': False,
    'facility_risks': False,
    'facility_types': lambda CSVRowDict: Base.metadata.tables['facility_types'].c['Fac_Type'] == CSVRowDict['Fac_Type'],
    'nel_sample_classes': lambda CSVRowDict: Base.metadata.tables['nel_sample_classes'].c['nel_column']==CSVRowDict['nel_column'],
    'existing_pollutant_concentrations': False, #uniqueness not imposed for records in this table.
    'wrs_pollutant_risks': False #DB schema does not impose uniqueness on records in this table
}

import SQLA_main as SQLA_main #import main SQLAlchemy functions



Clearing old DB


In [2]:
'''
Define other custom modules

'''
import mod_Base_BMP_Eval as BBMP_Eval
import mod_Combo_BMP_Eval as CBMP_Eval
import mod_EffluentLimit as EffLim
import mod_expression as Expr
import mod_importSpecial as importSpecial #special import functions are defined here
import mod_importCSV as importCSV #generic CSV importer ****IMPORTANT NOTE: function assumes csv in the utf-8-sig file format. weird things happen if its not in this format!!!


In [3]:
#import feasibillity questions, build feasibility expressions
importSpecial.importFeasibilityQuestionsCSV('Input_Files\\feasibility_test_questions.csv') 

#import base bmp information including:
  #1. imports definitions for cip costs, o&m costs, and BMP sizing to the expressions table
  #2. imports pollutant removal rates into pollutant_removal_rates table
  #3. creates a record in the base_bmps table using (1) and (2)
  #4. feasibility tests
importSpecial.importBaseBMPsCSV('Input_Files\\bmp_lego_piece.csv') 

#IMPORT BASIC FACILITY CHARS:
    #!!!!IMPORTANT!!!! This import must occur before other facility specific data is imported!
print ('\nImporting facility characteristics:')
importCSV.importCSV('Input_Files\\facility_chars.csv', unqTests)

#IMPORT PBP Appendix A1 data
print ('\nImporting PBP Appendix A1 data:')
importCSV.importCSV('Input_Files\\pbp_appxa1.csv', unqTests)

#IMPORT FACILITY RAINFALL EXTRACTED FROM http://rainfall.geography.hawaii.edu/downloads.html
print ('\nImporting Facility Rainfall Data:')
importCSV.importCSV('Input_Files\\FacilityRainfallData.csv', unqTests)

#IMPORT EFFLUENT LIMITS EXISTANCE FOR FACILITY TYPES: (either by Priority Based Plan, Table 3 or as City operational assignment)
#IF CSV HEADRS SETUP CORRECTLY, THEN THIS INSERTS NEL EXISTANCE DATA (0 OR 1) TO WRS_POLLUTANT TABLE 
#AND USES THE FACILITY_TYPE_HAS_NEL TO ASSOCIATE RECORD WITH FACILITY TYPE
print ('\nImporting Facility Type Has Effluent Limits:') #import into wrs_pollutant_risks table
importCSV.importCSV('Input_Files\\nel_exists_facility_types.csv', unqTests)

#IMPORT NEL CLASSIFICATION DATA (from PBP Appendix L)
print ('\nImporting NEL Classes')
importCSV.importCSV('Input_Files\\nel_pbp_appxl.csv', unqTests)

#IMPORT FACILITY RISKS:
print ('\nImporting Facility Risks')
#for future implementation:
    #The current process inserts fac risk and update existing_fac_char_id in Facility_chars table. this process thus creates
#dead records. a more sophisticated approach using sophisticated lambda function in unqTests would fix this
importCSV.importCSV('Input_Files\\facility_risks.csv', unqTests)

# #IMPORT FACILITY SAMPLING DATA
 #!!!IMPORTANT!!!! For now, we make none detects = 0 BUT this must be changed to detection limit, per DOH guidance.
print ('\nImporting Facilty Sampling data:')
importCSV.importCSV('Input_Files\\sample_data.csv', unqTests)


# for now, since we're developing, delete out all except 1st 2 facilities.
# n = 5
# session.query(ExPollConcs).filter(ExPollConcs.facility_id >n).delete(synchronize_session = False) #http://docs.sqlalchemy.org/en/latest/orm/query.html#sqlalchemy.orm.query.Query.delete
# session.query(Facility_Chars).filter(Facility_Chars.id >n).delete(synchronize_session = False) #http://docs.sqlalchemy.org/en/latest/orm/query.html#sqlalchemy.orm.query.Query.delete
# session.commit #we chose not to sync session so need to commit before proceeding to requery or else you may get unpredictable resutls

session.commit()
winsound.Beep(250,1000)

Reading csv for import to Feasibility Questions

Reading csv record: Feas-1

Reading csv record: Feas-2

Reading csv record: Feas-3

Reading csv record: Feas-4

Reading csv record: Feas-5

Reading csv record: Feas-6

Reading csv record: Feas-7

Reading csv record: Feas-8

Reading csv record: Feas-9

Reading csv record: Feas-10

Reading csv record: Feas-11

Reading csv record: Feas-12

Reading csv record: Feas-13

Reading csv record: Feas-14

Reading csv record: Feas-15

Reading csv record: Feas-16

Reading csv record: Feas-17

Reading csv record: Feas-18

Reading csv record: Feas-19

Reading csv record: Feas-20

Reading csv record: Feas-21
Reading csv for import to base bmp tables

Reading csv record: Hydrodynamic Separation
Reading pollutant removal rate info...
Linking feasibility tests w/ base bmp: 1
Removed:  0  old feasibility test defs for the base bmp
Added feasibility test def as record:  1
Added feasibility test def as record:  2
Added feasibility test def as record:  3
Added 


******Evaluating Base BMP feasibility at facilities.******
*****************************************************************
* Completed evaluating Base BMP feasibility                     *
3456  errors were encountered. Review output to identify location(s)
Hint: expression evaluation error lines are prefixed by: FAULT!!!! Error occured while evaluating expression:
*****************************************************************


In [5]:
'''
Enter estimated pollutant concentrations into database's existing pollutant concentration table for facilities without 
actual sampling data. Use 1 of 2 methods:

Method 1: Use maximum concentration value sampled for period 2013-2017
          This method is for Permit Table 1 facilities only
          Method assumes we have already entered sampling data for into the database's existing pollutant concentration table

Method 2: Use data from an EMC study.
          This method is for facilities that are not on Permit Table 1
          
          
Defined Global Vars:
    pd_infieldExtrema: dataframe holding maximum concentrations that were measured. *exception is phMin. we use lowest observed
        ###need to fill this out - typ all cells.
    
'''
#this is the list of pollutant constituants we're trying to address:
pollLS = ['tss', 'turbidity', 'p', 'n', 'nn', 'an', 'og', 'cu', 'zn', 'fe', 'phmin', 'phmax'] 


'''make a dataframe called pd_Concs to hold existing pollutant concentrations that 
were sampled in the field (the 'infield' sampling method)'''
q = session.query(ExPollConcs).filter(ExPollConcs.sample_method == 'infield')
pd_Concs = pd.read_sql(q.statement,session.bind) 

#Apply Method1 to all unsampled Table 1 facilities
#we need to get all Table 1 facilities that are not currently in the existing pollutant concentration table
#############################################################################################################
'''
To be sure we're starting fresh, let's remove any records in ExPollConcs that:
1. Were not obtained directly from field samples (i.e. sample_method != 'infield)
2. Were obtained from field samples, but are not Table 1 facilities (i.e. we shouldn't be looking at their  sample results)
'''
#delete all pollutant concentration table records that are not from infield sampling.
session.query(ExPollConcs).filter(ExPollConcs.sample_method != 'infield').delete(synchronize_session = False)
#delete all pollutant concentration table records that are not for Table 1 facilities
#for some reason bulk delete's not working. so let's use a loop to work around it.
for rec in session.query(ExPollConcs.id).filter(ExPollConcs.facility_id == Facility_Chars.id).filter(Facility_Chars.Permit_Table != 'Table 1'):
    session.query(ExPollConcs).filter(ExPollConcs.id == rec[0]).delete(synchronize_session = False)

    
#build pd_infieldExtreama by making a dictionary of maximum sample results for each constiuent
dict_extrema = {'c_' + Constituent: pd_Concs.loc[:,'c_' + Constituent].max() for Constituent in pollLS}
dict_extrema['c_phmin'] = pd_Concs.loc[:,'c_phmin'].min() #phMin is exception to above. we want min. phMin value
#use dictionary to build pd_infieldExtrema dataframe
pd_infieldExtrema = pd.DataFrame([dict_extrema])
display(pd_infieldExtrema)

#now build query that identifies all Table 1 facilities that are not in ExPollConcs
subq = session.query(ExPollConcs.facility_id.distinct()).order_by(ExPollConcs.facility_id).all()
ls_sq = [i[0] for i in subq if i[0] is not None] #list comprehension to produce list of all facility_id in ExPollConcs table
#get list of Table 1 facilities not in ExPollConcs:
tpl_q = session.query(Facility_Chars.id).filter(Facility_Chars.Permit_Table == 'Table 1').filter(Facility_Chars.id.notin_(ls_sq)).all()
ls_FacIDs = [i[0] for i in tpl_q] #write query tuple to list    
#make a list of Table 1 facs not in ExPollConcs (a list of dicts). also include extrema conc. values.  
ls_dict_pd = [{**{'facility_id': FacID, 'sample_method': 'sim_MaxType', 'sample_date':'12/31/2016'}, **dict_extrema} for FacID in ls_FacIDs]
#write list to database:
ExPollConcs_meta = Base.metadata.tables['existing_pollutant_concentrations']
ExPollConcs_id_meta = ExPollConcs_meta.c['id']
for dict_temp in ls_dict_pd:
    SQLA_main.insertRec(ExPollConcs_meta,dict_temp)
session.commit()
#for future implementation: write dict -> dataframe -> db(using sqla):
    # pd_temp.to_sql('existing_pollutant_concentrations', engine, if_exists='append', index = False)
    #http://docs.sqlalchemy.org/en/latest/faq/performance.html#i-m-inserting-400-000-rows-with-the-orm-and-it-s-really-slow
    #https://stackoverflow.com/questions/31997859/bulk-insert-a-pandas-dataframe-using-sqlalchemy
q = session.query(ExPollConcs)
pd.read_sql(q.statement,session.bind) 


c_an   c_cu     c_fe    c_n  c_nn  c_og   c_p  c_phmax  c_phmin   c_tss  \
0  10.5  105.0  19500.0  27.25  3.68  59.0  10.3      8.7     6.01  2010.0   

   c_turbidity   c_zn  
0       4181.0  730.0

id  facility_id sample_method sample_point_name sample_date   c_tss  \
0      4          103       infield           CAESHAL   4/20/2017     NaN   
1      5          103       infield           CAESHAL   2/11/2017     NaN   
2      6          103       infield           CAESHAL   12/8/2016    30.0   
3      7          103       infield           CAESHAL   6/17/2016    23.0   
4      8          103       infield           CAESHAL    5/2/2014    42.5   
5      9          103       infield           CAESHAL    3/9/2013    53.5   
6     10          103       infield           CAESHAL   1/27/2013     NaN   
7     11          107       infield          CWCSMHAL    5/2/2014    30.5   
8     12          107       infield          CWCSMHAL   1/27/2013   233.0   
9     13          106       infield           CDRMHAL   4/20/2017     NaN   
10    14          106       infield           CDRMHAL   2/11/2017     NaN   
11    15          106       infield           CDRMHAL   12/8/2016    28.0   
12    16          106       infield           CDRMHAL   6/17/2016    70.0   
13    17          106       infield           CDRMHAL    5/2/2014     8.0   
14    18          106       infield           CDRMHAL   1/27/2013   214.0   
15    20          108       infield         CWTPHON-2   2/11/2017   677.0   
16    21          108       infield         CWTPHON-1   2/11/2017   910.0   
17    22          108       infield           CWTPHON   4/14/2013   303.0   
18    23           92       infield           CDRMKAI   4/12/2015     5.0   
19    24          109       infield         CWTPKAI-2    2/6/2017     NaN   
20    25          109       infield         CWTPKAI-1    2/6/2017   680.0   
21    26          109       infield         CWTPKAI-2   12/1/2016   272.0   
22    27          109       infield         CWTPKAI-1   6/17/2016   151.0   
23    28          109       infield         CWTPKAI-2   5/25/2016   157.0   
24    29          109       infield         CWTPKAI-1    2/3/2015   119.0   
25    30          109       infield         CWTPKAI-2    2/3/2015   155.0   
26    31          109       infield         CWTPKAI-1    3/9/2013    10.5   
27    33           82       infield          CCSLKALA   2/11/2017   255.0   
28    35            1       infield          CDTSKALI   4/19/2017   122.0   
29    36            1       infield          CDTSKALI   2/11/2017    59.0   
..   ...          ...           ...               ...         ...     ...   
77    94          110       infield           CWTPSAN   2/20/2015    16.0   
78    95          110       infield           CWTPSAN   3/24/2013     6.0   
79    97           12       infield           CRCCWAH   3/22/2013    62.5   
80    98          111       infield           CWTPWAH   4/20/2017     NaN   
81    99          111       infield           CWTPWAH   1/21/2017    13.0   
82   100          111       infield           CWTPWAH   11/5/2015     6.0   
83   101          112       infield          CWTPWAIA   2/11/2017   142.0   
84   106           13       infield          CRCCWAIM   4/28/2017   280.0   
85   107           13       infield          CRCCWAIM    2/6/2017    80.0   
86   108           13       infield          CRCCWAIM   3/22/2013  2010.0   
87   109           83       infield        CCSLWAIP-1   4/20/2017    72.0   
88   110          240       infield           CDPRWAI   9/11/2015   293.0   
89   111            6   sim_MaxType              None  12/31/2016  2010.0   
90   112            9   sim_MaxType              None  12/31/2016  2010.0   
91   113           10   sim_MaxType              None  12/31/2016  2010.0   
92   114           11   sim_MaxType              None  12/31/2016  2010.0   
93   115           14   sim_MaxType              None  12/31/2016  2010.0   
94   116           15   sim_MaxType              None  12/31/2016  2010.0   
95   117           34   sim_MaxType              None  12/31/2016  2010.0   
96   118           41   sim_MaxType              None  12/31/2016  2010.0   
97

In [6]:
#Estimate Pollutant Effluent Limits
'''
Estimate the Numeric Effluent Limits (NELs) for each facility.
Return wet and dry season NELs in 2 separate dataframes:
    pd_FacsNELs_Wet & pd_FacsNELs_Dry
Estimate NELs using the EffLim module's GetNELs function call.
 The GetNELs function call will differentiate between wet and dry season limits
 (if limits are the same between wet & dry season, then the same limit will be placed into the wet and dry
  dataframes.)
 The GetNEls function calculates a pollutant constituent NEL using this formula:
    NEL = fTypeHas_NEL * SampleClass_NEL
    Where:
      fTypeHas_NEL is a [0,1] value from PBP Table 3, based on facility type (stored in SQLA_DB_facility_type_has_nel)
      SampleClass_NEL is pollutant concentration based on facility's sample class, based on PBP Appendix L
'''
pd_FacsNELs_Wet, pd_FacsNELs_Dry = pd.DataFrame(),  pd.DataFrame() #initialize wet and dry season nel dataframes 
for recFac in session.query(Facility_Chars): #do the following for each facility:
    
    wet,dry = EffLim.GetNELs(recFac,False) #Get Wed & Dry NELs by calculating: NEL = fTypeHas_NEL * SampleClass_NEL
#     if wet is not None:
    pd_FacsNELs_Wet = pd.concat([pd_FacsNELs_Wet, wet]) #write wet NELs to pd_FacsNELs_Wet
#     if dry is not None:
    pd_FacsNELs_Dry = pd.concat([pd_FacsNELs_Dry, dry]) #write dry NELs to pd_FacsNELs_Dry

print('Wet NELs:')
display(pd_FacsNELs_Wet)
print('Dry NELs:')
display(pd_FacsNELs_Dry)

Wet NELs:


wrs_tss  wrs_turbidity  wrs_p  wrs_n  wrs_nn  wrs_an  wrs_og  \
Facility_ID                                                                 
1               50.0          15.00   0.10   0.52   0.180     NaN    15.0   
2               50.0          15.00   0.10   0.52   0.180     NaN    15.0   
3                NaN            NaN    NaN    NaN     NaN     NaN    15.0   
4               50.0          15.00   0.10   0.52   0.180     NaN    15.0   
5               50.0          15.00   0.10   0.52   0.180     NaN    15.0   
6               50.0          15.00   0.10   0.52   0.180     NaN    15.0   
7               50.0          15.00   0.10   0.52   0.180     NaN    15.0   
8                NaN            NaN    NaN    NaN     NaN     NaN    15.0   
9                NaN           0.50   0.03   0.18   0.010  0.0050    15.0   
10              50.0          15.00   0.10   0.52   0.180     NaN    15.0   
11              50.0          15.00   0.10   0.52   0.180     NaN    15.0   
12              50.0          15.00   0.10   0.52   0.180     NaN    15.0   
13              50.0          15.00   0.10   0.52   0.180     NaN    15.0   
14              50.0          15.00   0.10   0.52   0.180     NaN    15.0   
15              50.0          15.00   0.10   0.52   0.180     NaN    15.0   
16               NaN           3.00   0.05   0.35   0.025  0.0100    15.0   
17               NaN           8.00   0.13   0.55   0.040  0.0200    15.0   
18              50.0          15.00   0.10   0.52   0.180     NaN    15.0   
19               NaN            NaN    NaN    NaN     NaN     NaN    15.0   
20              50.0          15.00   0.10   0.52   0.180     NaN    15.0   
21               NaN            NaN    NaN    NaN     NaN     NaN    15.0   
22               NaN           1.25   0.04   0.25   0.014  0.0085    15.0   
23               NaN           1.25   0.04   0.25   0.014  0.0085    15.0   
24              50.0          15.00   0.10   0.52   0.180     NaN    15.0   
25              50.0          15.00   0.10   0.52   0.180     NaN    15.0   
26              50.0          15.00   0.10   0.52   0.180     NaN    15.0   
27              50.0          15.00   0.10   0.52   0.180     NaN    15.0   
28              50.0          15.00   0.10   0.52   0.180     NaN    15.0   
29              50.0          15.00   0.10   0.52   0.180     NaN    15.0   
30               NaN           8.00   0.13   0.55   0.040  0.0200    15.0   
...              ...            ...    ...    ...     ...     ...     ...   
92              50.0          15.00   0.10   0.52   0.180     NaN    15.0   
93               NaN           3.00   0.05   0.35   0.020  0.0130    15.0   
94               NaN            NaN    NaN    NaN     NaN     NaN    15.0   
95               NaN           1.25   0.04   0.25   0.014  0.0085    15.0   
96               NaN           3.00   0.05   0.35   0.020  0.0130    15.0   
97              50.0          15.00   0.10   0.52   0.180     NaN    15.0   
98              50.0          15.00   0.10   0.52   0.180     NaN    15.0   
99              50.0          15.00   0.10   0.52   0.180     NaN    15.0   
100             50.0          15.00   0.10   0.52   0.180     NaN    15.0   
101              NaN            NaN    NaN    NaN     NaN     NaN    15.0   
102              NaN           3.00   0.05   0.35   0.025  0.0100    15.0   
103             50.0          15.00   0.10   0.52   0.180     NaN    15.0   
104              NaN            NaN    NaN    NaN     NaN     NaN    15.0   
105              NaN           3.00   0.05   0.35   0.020  0.0130    15.0   
106             50.0          15.00   0.10   0.52   0.180     NaN    15.0   
107             50.0          15.00   0.10   0.52   0.180     NaN    15.0   
108             50.0          15.00   0.10   0.52   0.180     NaN    15.0   
109              NaN            NaN    NaN    NaN     NaN     NaN    15.0   
110              NaN           3.00   0.05   0.35   0.020  0.0130    15.0   
111      

Dry NELs:


wrs_tss  wrs_turbidity  wrs_p  wrs_n  wrs_nn  wrs_an  wrs_og  \
Facility_ID                                                                 
1               30.0           5.50   0.06   0.38   0.090     NaN    15.0   
2               30.0           5.50   0.06   0.38   0.090     NaN    15.0   
3                NaN            NaN    NaN    NaN     NaN     NaN    15.0   
4               30.0           5.50   0.06   0.38   0.090     NaN    15.0   
5               30.0           5.50   0.06   0.38   0.090     NaN    15.0   
6               30.0           5.50   0.06   0.38   0.090     NaN    15.0   
7               30.0           5.50   0.06   0.38   0.090     NaN    15.0   
8                NaN            NaN    NaN    NaN     NaN     NaN    15.0   
9                NaN           0.50   0.03   0.18   0.010  0.0050    15.0   
10              30.0           5.50   0.06   0.38   0.090     NaN    15.0   
11              30.0           5.50   0.06   0.38   0.090     NaN    15.0   
12              30.0           5.50   0.06   0.38   0.090     NaN    15.0   
13              30.0           5.50   0.06   0.38   0.090     NaN    15.0   
14              30.0           5.50   0.06   0.38   0.090     NaN    15.0   
15              30.0           5.50   0.06   0.38   0.090     NaN    15.0   
16               NaN           3.00   0.05   0.35   0.025  0.0100    15.0   
17               NaN           8.00   0.13   0.55   0.040  0.0200    15.0   
18              30.0           5.50   0.06   0.38   0.090     NaN    15.0   
19               NaN            NaN    NaN    NaN     NaN     NaN    15.0   
20              30.0           5.50   0.06   0.38   0.090     NaN    15.0   
21               NaN            NaN    NaN    NaN     NaN     NaN    15.0   
22               NaN           1.25   0.04   0.25   0.014  0.0085    15.0   
23               NaN           1.25   0.04   0.25   0.014  0.0085    15.0   
24              30.0           5.50   0.06   0.38   0.090     NaN    15.0   
25              30.0           5.50   0.06   0.38   0.090     NaN    15.0   
26              30.0           5.50   0.06   0.38   0.090     NaN    15.0   
27              30.0           5.50   0.06   0.38   0.090     NaN    15.0   
28              30.0           5.50   0.06   0.38   0.090     NaN    15.0   
29              30.0           5.50   0.06   0.38   0.090     NaN    15.0   
30               NaN           8.00   0.13   0.55   0.040  0.0200    15.0   
...              ...            ...    ...    ...     ...     ...     ...   
92              30.0           5.50   0.06   0.38   0.090     NaN    15.0   
93               NaN           3.00   0.05   0.35   0.020  0.0130    15.0   
94               NaN            NaN    NaN    NaN     NaN     NaN    15.0   
95               NaN           1.25   0.04   0.25   0.014  0.0085    15.0   
96               NaN           3.00   0.05   0.35   0.020  0.0130    15.0   
97              30.0           5.50   0.06   0.38   0.090     NaN    15.0   
98              30.0           5.50   0.06   0.38   0.090     NaN    15.0   
99              30.0           5.50   0.06   0.38   0.090     NaN    15.0   
100             30.0           5.50   0.06   0.38   0.090     NaN    15.0   
101              NaN            NaN    NaN    NaN     NaN     NaN    15.0   
102              NaN           3.00   0.05   0.35   0.025  0.0100    15.0   
103             30.0           5.50   0.06   0.38   0.090     NaN    15.0   
104              NaN            NaN    NaN    NaN     NaN     NaN    15.0   
105              NaN           3.00   0.05   0.35   0.020  0.0130    15.0   
106             30.0           5.50   0.06   0.38   0.090     NaN    15.0   
107             30.0           5.50   0.06   0.38   0.090     NaN    15.0   
108             30.0           5.50   0.06   0.38   0.090     NaN    15.0   
109              NaN            NaN    NaN    NaN     NaN     NaN    15.0   
110              NaN           3.00   0.05   0.35   0.020  0.0130    15.0   
111      

In [7]:
import pandas as pd
import numpy as np
import math
import datetime

In [8]:
#let's work on calculating exceedance values

import mod_EffluentLimit as EffLim

#get all sampling data
q = session.query(ExPollConcs.id, ExPollConcs.facility_id.label('Facility_ID'), ExPollConcs.sample_date, 
        ExPollConcs.c_tss,
        ExPollConcs.c_turbidity,
        ExPollConcs.c_p,
        ExPollConcs.c_n,
        ExPollConcs.c_nn,
        ExPollConcs.c_an,
        ExPollConcs.c_og,
        ExPollConcs.c_cu,
        ExPollConcs.c_zn,
        ExPollConcs.c_fe,
        ExPollConcs.c_phmin,
        ExPollConcs.c_phmax  
         ).order_by(ExPollConcs.facility_id) #.filter(ExPollConcs.facility_id == FacID)
pd_Concs = pd.read_sql(q.statement,session.bind) 
#tidy up the sampling data
from datetime import datetime
pd_Concs['sample_date'] = pd.to_datetime(pd_Concs['sample_date'], format="%m/%d/%Y")
pd_Concs = pd_Concs.applymap(lambda x: float('nan') if x is None else x) #assign NaN values to any None element

#for each pollutant constituent, do the Exceedance Calculation = max(0,(Constituent Concentration - NEL))
# if no exceedance, then report 0. report NaN sample result is NaN
pd_FacExceedances = pd.DataFrame() #make an empty dataframe.  we will append to it.
for Facs in session.query(Facility_Chars.id).order_by(Facility_Chars.id):
    FacID = Facs[0]
    if (FacID in pd_Concs.Facility_ID.values) and (FacID in pd_FacsNELs_Dry.index) and (FacID in pd_FacsNELs_Wet.index):
        pd_temp = pd_Concs.loc[pd_Concs['Facility_ID'] == FacID] #slice facility id rows into a temp dataframe
        pd_temp.is_copy = False #acknowledge that pd_temp is NOT a copy of pd_Concs. but intended to be treated as new dataframe
        for Constituent in pollLS:
            pd_temp['c_' + Constituent]  = pd_Concs.apply(lambda row: #for each row in pd_Concs:
                                     EffLim.ExceedanceCalc(row, Constituent, FacID, pd_FacsNELs_Wet, pd_FacsNELs_Dry, True), axis = 1)

        pd_FacExceedances = pd.concat([pd_FacExceedances,pd_temp])
print('Concentrations in excess of wet/dry season NELs')
display(pd_FacExceedances)

Concentrations in excess of wet/dry season NELs


id  Facility_ID sample_date   c_tss  c_turbidity     c_p     c_n   c_nn  \
0     35            1  2017-04-19    72.0          NaN     NaN   1.910    NaN   
1     36            1  2017-02-11     9.0          0.0   0.000   0.060    NaN   
2     37            1  2016-12-04    30.0         56.2   0.200   0.390  0.000   
3     38            1  2016-06-17    53.0         75.9   0.190   0.560  0.110   
4     39            1  2015-02-20     0.0          2.5   0.076   1.310  0.010   
5     40            1  2014-04-13     0.0          0.0   0.144   1.517  0.067   
6     41            1  2013-03-09   113.0          9.4   0.055   0.679  0.059   
7     80            2  2017-04-20     NaN          0.0     NaN   0.260    NaN   
8     81            2  2017-01-21     0.0         16.0   0.000   1.060  0.000   
9     82            2  2016-05-05     0.0          0.0   0.006   2.292  0.122   
10    83            2  2014-06-03     0.0          0.0   0.015   0.756  0.026   
11    84            2  2013-03-09    30.0          0.0   0.067   0.241  0.000   
12    51            3  2017-04-28     NaN          NaN     NaN     NaN    NaN   
13    52            3  2016-12-01     NaN          NaN     NaN     NaN    NaN   
14    53            3  2015-02-14     NaN          NaN     NaN     NaN    NaN   
15    54            3  2013-03-14     NaN          NaN     NaN     NaN    NaN   
16    60            4  2013-04-24   197.0          0.0   0.655   0.829  0.000   
17    62            5  2017-01-24   199.0         31.0   2.200  15.330  0.000   
18    63            5  2016-12-08   190.0        752.0   0.037  26.730  0.000   
19    64            5  2016-05-25    59.5          7.7   2.860  14.162  0.052   
20    65            5  2013-03-09    82.0         38.0   2.430   0.000  0.000   
21   111            6  2016-12-31  1960.0       4166.0  10.200  26.730  3.500   
22    87            7  2014-06-03     0.0         12.8   0.000   0.323  0.043   
23    50            8  2013-03-07     NaN          NaN     NaN     NaN    NaN   
24   112            9  2016-12-31     NaN       4180.5  10.270  27.070  3.670   
25   113           10  2016-12-31  1960.0       4166.0  10.200  26.730  3.500   
26   114           11  2016-12-31  1960.0       4166.0  10.200  26.730  3.500   
27    97           12  2013-03-22    12.5         19.4   0.351   2.127  0.197   
28   106           13  2017-04-28   230.0        121.0   1.700  20.580  1.320   
29   107           13  2017-02-06    30.0         58.0   1.100   4.380  0.540   
..   ...          ...         ...     ...          ...     ...     ...    ...   
76    18          106  2013-01-27   164.0        174.0   0.368   0.622  0.000   
77    11          107  2014-05-02     0.5         11.9   0.101   0.772  0.002   
78    12          107  2013-01-27   183.0        225.0   0.458   0.723  0.023   
79    20          108  2017-02-11   627.0        705.0   1.400   1.130  0.940   
80    21          108  2017-02-11   860.0         90.0   0.200   0.560  0.340   
81    22          108  2013-04-14   253.0          1.0   0.774   1.367  0.000   
82    24          109  2017-02-06     NaN          NaN     NaN     NaN    NaN   
83    25          109  2017-02-06     NaN          NaN     NaN     NaN    NaN   
84    26          109  2016-12-01     NaN          NaN     NaN     NaN    NaN   
85    27          109  2016-06-17     NaN          NaN     NaN     NaN    NaN   
86    28          109  2016-05-25     NaN          NaN     NaN     NaN    NaN   
87    29          109  2015-02-03     NaN          NaN     NaN     NaN    NaN   
88    30          109  2015-02-03     NaN          NaN     NaN     NaN    NaN   
89    31          109  2013-03-09     NaN          NaN     NaN     NaN    NaN   
90    88          110  2017-04-19     NaN         42.0   0.430   1.330  0.550   
91    89          110  2017-04-19     NaN         44.0   0.430   1.100  0.450   
92    90          110  2017-01-24     NaN         87.0   0.450   1.840  0.500   
93    91          110  2017-01-24    

In [22]:
'''
CALCULATE AGE FACTOR WEIGHTED AVERAGE FOR EACH CONSTITUENT:

Age factor acknowledges fact that more recent samples are a better representation of facility pollutant discharge 
(i.e. sampling data) and housekeeping-operations (i.e. inspections) realities. But, historic data as a whole also tells part 
of story (i.e. we want to dampen whipsaw effects that may occur if we only considered most recent data).

AF = exp(-SampleRank)
SampleRank = Newest sample = 1
              Second Newest sample = 2
              ...
              nth Newest Sample = n (out of n samples)
'''
#calculate age factor weighted averages for each constituent of each facility.
#write these averages into a dataframe called pd_AFWFacExceedances
#use exceedance values in pd_FacExceedances
#we will process each facility separately. The results will be added to the pd_AFWFacExceedances dataframe
pd_AFWFacExceedances = pd.DataFrame() #make an empty dataframe.  we will append to it.
ShowCalculations = False
UnqFacIDs = pd_FacExceedances.Facility_ID.unique() #get unique facility IDs
for FacID in UnqFacIDs:
    pd_temp = pd_FacExceedances.loc[pd_FacExceedances['Facility_ID'] == FacID].set_index('sample_date') #write current fac's data to temp df
    pd_AFWFacExceedances = pd.concat([pd_AFWFacExceedances,EffLim.Calc_AFWtdExceedances(pd_temp,pollLS,ShowCalculations)]) #calculate age factor weighted averages
print ('Age Factor Weighted Averages:')
display(pd_AFWFacExceedances)

Age Factor Weighted Averages:


Facility_ID  AFWtd_c_tss  AFWtd_c_turbidity  AFWtd_c_p  AFWtd_c_n  \
0            1    52.064543          19.729711   0.067223   1.296165   
0            2     0.961758           3.745947   0.004876   0.637805   
0            3          NaN                NaN        NaN        NaN   
0            4   197.000000           0.000000   0.655000   0.829000   
0            5   180.960566         199.986459   1.752511  17.437221   
0            6  1960.000000        4166.000000  10.200000  26.730000   
0            7     0.000000          12.800000   0.000000   0.323000   
0            8          NaN                NaN        NaN        NaN   
0            9          NaN        4180.500000  10.270000  27.070000   
0           10  1960.000000        4166.000000  10.200000  26.730000   
0           11  1960.000000        4166.000000  10.200000  26.730000   
0           12    12.500000          19.400000   0.351000   2.127000   
0           13   336.807197         469.755775   1.674704  15.243243   
0           14  1960.000000        4166.000000  10.200000  26.730000   
0           15  1960.000000        4166.000000  10.200000  26.730000   
0           34  1960.000000        4166.000000  10.200000  26.730000   
0           41          NaN        4173.000000  10.170000  26.700000   
0           81    72.443042          99.914697   0.467443   1.707622   
0           82          NaN                NaN        NaN        NaN   
0           83          NaN          13.000000   0.148000   0.940000   
0           84  1960.000000        4166.000000  10.200000  26.730000   
0           85          NaN                NaN        NaN        NaN   
0           92     0.000000           0.000000   0.000000   0.496000   
0           93          NaN        4178.000000  10.250000  26.900000   
0           94          NaN                NaN        NaN        NaN   
0           95          NaN        4179.750000  10.260000  27.000000   
0           96          NaN        4178.000000  10.250000  26.900000   
0           97    43.568510          30.576732   1.132181   1.131914   
0           98  1960.000000        4166.000000  10.200000  26.730000   
0           99    16.559240           4.868334   0.504841   4.083533   
0          100     0.000000           0.000000   0.258000   2.260000   
0          101          NaN                NaN        NaN        NaN   
0          102          NaN        4178.000000  10.250000  26.900000   
0          103     1.201509           7.864125   0.152872   2.174979   
0          104          NaN                NaN        NaN        NaN   
0          105          NaN        4178.000000  10.250000  26.900000   
0          106    14.732924          16.599986   0.691190   0.829885   
0          107    49.581809          69.211417   0.197012   0.758822   
0          108   696.752557         515.665239   1.231643   1.193739   
0          109          NaN                NaN        NaN        NaN   
0          110          NaN          49.764373   1.282323   1.405886   
0          111          NaN                NaN        NaN        NaN   
0          112          NaN          15.400000   0.860000   3.290000   
0          113  1960.000000        4166.000000  10.200000  26.730000   
0          114    28.238849          23.715705   0.198266   0.706055   

   AFWtd_c_nn  AFWtd_c_an  AFWtd_c_og  AFWtd_c_cu  AFWtd_c_zn    AFWtd_c_fe  \
0    0.029425         NaN    0.000000         NaN         NaN           NaN   
0    0.031165         NaN    0.000000         NaN         NaN           NaN   
0         NaN         NaN    0.000000         NaN         NaN           NaN   
0    0.000000         NaN    0.000000         NaN         NaN           NaN   
0    0.004532         NaN    1.421297         NaN         NaN           NaN   
0    3.500000         NaN   44.000000         NaN         NaN           NaN   
0    0.043000         NaN    0.000000         NaN         NaN           NaN   
0         NaN         NaN    0.000000         NaN         

45


In [10]:
#calculate monthly runoff volumes in cubic feet for each facility.
#hold data in dataframe called pd_RunoffVols
import calendar

#get facility imperviousness and area. order by Facility_ID so it's given in same order as monthly rain data dataframe
q_facDat = session.query(Facility_Chars.id.label('Facility_ID'), 
                         Facility_Chars.Fac_Area, 
                         Facility_Chars.Imperv.label('Imperv')).order_by('Facility_ID')
pd_facDat = pd.read_sql(q_facDat.statement,session.bind)

#get monthly rain data for each facility. order by facility_id so order matches facility data dataframe
q_rain = session.query(Facility_Chars.id.label('Facility_ID'), Facility_Monthly_Rain).filter(
    Facility_Chars.facility_monthly_rain_id == Facility_Monthly_Rain.id).order_by('Facility_ID')
pd_rainDat = pd.read_sql(q_rain.statement,session.bind)

#create a new dataframe to hold rain volumes
pd_RunoffVols = pd_facDat.loc[:,['Facility_ID']] #put facilities into the new dataframe
#now calculate volumes for each month:
for mo in range(1,13):
    pd_RunoffVols[calendar.month_name[mo]] = pd.DataFrame(pd_facDat['Fac_Area'] * pd_facDat['Imperv'] * pd_rainDat[calendar.month_name[mo]]/12)
#add monthlys together to get annual volume
pd_RunoffVols['Annual_Volume'] = pd_RunoffVols[[calendar.month_name[mo] for mo in range (1,13)]].sum(axis = 1)

display(pd_RunoffVols)

Facility_ID        January       February          March          April  \
0              1  303546.250000  344735.416667  362643.750000  256984.583333   
1              2  338916.586546  246040.406579  241152.186581  101837.916631   
2              3  110527.319917  106519.649920  112214.759916   75723.869943   
3              4   21601.669993   18133.109161   16552.499161    9439.754164   
4              5   50545.681653   57766.493317   56824.648318   36261.032490   
5              6   15974.190828   18427.906661   18127.451661   11667.669163   
6              7   10058.130000    7407.150000    7069.280000    2858.900000   
7              8   31502.679978   29712.754979   32397.642477   21658.092485   
8              9    9823.037518    8034.365015    6568.240012    3782.602507   
9             10    7359.825004    4947.100003    3973.900002    2108.600001   
10            11   38203.193333   36076.720000   44289.306667   30210.586667   
11            12   15778.000002   11261.833335   11004.583335    6116.833334   
12            13   10628.624992    8401.674994    9758.089992    4534.879997   
13            14    5740.670832    3474.224166    3354.937499    1520.905000   
14            15    4947.973337    3860.506670    3806.133336    1644.793335   
15            16    9943.041593    7764.165194    9544.092393    5892.172796   
16            17    3686.471831    2784.256357    2629.036490    1096.240308   
17            18   53844.879604   33653.049752   40716.035503   21022.769352   
18            19    3815.469177    2877.239051    4020.986252    2528.753576   
19            20    8321.346913    6368.778897    8832.421763    7757.854130   
20            21    5387.240724    5977.084599    5387.240724    3788.108441   
21            22   39352.347381   24823.730133   20301.577744   12892.945108   
22            23   52497.744370   48245.590621   42685.081871   20115.958123   
23            24    8061.145988    8891.241112   10108.713962    8983.473904   
24            25   11225.985481   10110.989181   14292.225308    8261.108955   
25            26    9456.558647    7778.121831    9006.246330    5874.528856   
26            27   13384.363069    7531.143068   10457.753068    5306.919467   
27            28    7193.151073    8000.020048    9098.735248    7897.015498   
28            29    7272.970203    5877.276977    7580.278803    6504.698702   
29            30    7089.868296    5606.305547    5653.154897    3279.454498   
..           ...            ...            ...            ...            ...   
210          211            NaN            NaN            NaN            NaN   
211          212            NaN            NaN            NaN            NaN   
212          213            NaN            NaN            NaN            NaN   
213          214            NaN            NaN            NaN            NaN   
214          215            NaN            NaN            NaN            NaN   
215          216            NaN            NaN            NaN            NaN   
216          217            NaN            NaN            NaN            NaN   
217          218            NaN            NaN            NaN            NaN   
218          219            NaN            NaN            NaN            NaN   
219          220            NaN            NaN            NaN            NaN   
220          221            NaN            NaN            NaN            NaN   
221          222            NaN            NaN            NaN            NaN   
222          223            NaN            NaN            NaN            NaN   
223          224            NaN            NaN            NaN            NaN   
224          225            NaN            NaN            NaN            NaN   
225          226            NaN            NaN            NaN            NaN   
226          227            NaN            NaN            NaN            NaN   
227          228            NaN            NaN            NaN            NaN   
228  

In [11]:
'''
CALCULATE EXISTING RAW POLLUTANT EXCEEDANCE POTENTIAL SCORES (exPEP_raw)
exPEP_raw = AFWtd Exceedance * Annual Runoff Volume (cu. ft)

'''
# #make pd_AnnRunoffVols dataframe of just annual runoff volumes:
# pd_AnnRunoffVols = pd_RunoffVols[pd_RunoffVols.columns.drop([calendar.month_name[mo] for mo in range (1,13)])]

#initialize pd_exPEP_raw dataframe w/ Facility_IDs from pd_AFWFacExceedances
pd_exPEP_raw = pd_AFWFacExceedances.loc[:,['Facility_ID']]
pd_exPEP_raw.reset_index(drop=True,inplace=True)

#calculate raw pep scores:
def _HELPER_calc_PEP_raw(row, Constituent):
    #HELPER function to calculate PEP_raw
    #include .values[0] to retrieve values from dataframe series (dunno why we're recieveing series and not val)
    AnnRunoffVol = pd_RunoffVols.loc[pd_RunoffVols['Facility_ID']==row['Facility_ID'],'Annual_Volume'].values[0]
    AFWFacExceedVal = pd_AFWFacExceedances.loc[pd_AFWFacExceedances['Facility_ID']==row['Facility_ID'],'AFWtd_c_'+Constituent].values[0]
    return  AFWFacExceedVal * AnnRunoffVol
#for each facility in pd_exPEP_raw, calculate PEP_Raw SCORE for each pollutant constituent in the pollLS LIST:
for Constituent in pollLS:
    pd_exPEP_raw['PEP_raw_' + Constituent] = pd_exPEP_raw.apply(lambda row: 
                                           _HELPER_calc_PEP_raw(row,Constituent), axis = 1)
display(pd_exPEP_raw)


Facility_ID   PEP_raw_tss  PEP_raw_turbidity     PEP_raw_p     PEP_raw_n  \
0             1  1.589724e+08       6.024212e+07  2.052564e+05  3.957671e+06   
1             2  2.297361e+06       8.947981e+06  1.164828e+04  1.523531e+06   
2             3           NaN                NaN           NaN           NaN   
3             4  3.033362e+07       0.000000e+00  1.008554e+05  1.276476e+05   
4             5  8.766133e+07       9.687790e+07  8.489555e+05  8.446978e+06   
5             6  3.024941e+08       6.429543e+08  1.574204e+06  4.125340e+06   
6             7  0.000000e+00       8.912283e+05  0.000000e+00  2.248959e+04   
7             8           NaN                NaN           NaN           NaN   
8             9           NaN       2.658819e+08  6.531772e+05  1.721666e+06   
9            10  8.516068e+07       1.810099e+08  4.431831e+05  1.161400e+06   
10           11  7.296678e+08       1.550916e+09  3.797251e+06  9.951031e+06   
11           12  1.386649e+06       2.152079e+06  3.893710e+04  2.359522e+05   
12           13  2.559726e+07       3.570131e+07  1.272771e+05  1.158482e+06   
13           14  6.666276e+07       1.416924e+08  3.469184e+05  9.091304e+05   
14           15  6.239775e+07       1.326271e+08  3.247230e+05  8.509652e+05   
15           34  5.638667e+08       1.198504e+09  2.934408e+06  7.689876e+06   
16           41           NaN       2.750512e+09  6.703262e+06  1.759853e+07   
17           81  0.000000e+00       0.000000e+00  0.000000e+00  0.000000e+00   
18           82           NaN                NaN           NaN           NaN   
19           83           NaN       1.620933e+06  1.845370e+04  1.172060e+05   
20           84  2.181791e+08       4.637419e+08  1.135422e+06  2.975473e+06   
21           85           NaN                NaN           NaN           NaN   
22           92  0.000000e+00       0.000000e+00  0.000000e+00  2.248371e+05   
23           93           NaN       2.968967e+09  7.283846e+06  1.911565e+07   
24           94           NaN                NaN           NaN           NaN   
25           95           NaN       1.526735e+09  3.747665e+06  9.862276e+06   
26           96           NaN       6.019064e+08  1.476673e+06  3.875367e+06   
27           97  1.793600e+07       1.258763e+07  4.660887e+05  4.659791e+05   
28           98  1.392570e+09       2.959922e+09  7.247048e+06  1.899153e+07   
29           99  2.421862e+06       7.120152e+05  7.383521e+04  5.972347e+05   
30          100  0.000000e+00       0.000000e+00  4.986590e+04  4.368098e+05   
31          101           NaN                NaN           NaN           NaN   
32          102           NaN       1.778818e+09  4.364023e+06  1.145290e+07   
33          103  1.704068e+06       1.115348e+07  2.168143e+05  3.084714e+06   
34          104           NaN                NaN           NaN           NaN   
35          105           NaN       4.912121e+08  1.205104e+06  3.162663e+06   
36          106  1.620177e+07       1.825497e+07  7.600998e+05  9.126226e+05   
37          107  2.139957e+07       2.987173e+07  8.503066e+04  3.275085e+05   
38          108  1.350819e+09       9.997382e+08  2.387829e+06  2.314344e+06   
39          109           NaN                NaN           NaN           NaN   
40          110           NaN       1.259836e+08  3.246333e+06  3.559146e+06   
41          111           NaN                NaN           NaN           NaN   
42          112           NaN       1.086004e+07  6.064696e+05  2.320099e+06   
43          113  0.000000e+00       0.000000e+00  0.000000e+00  0.000000e+00   
44          114  0.000000e+00       0.000000e+00  0.000000e+00  0.000000e+00   

      PEP_raw_nn    PEP_raw_an    PEP_raw_og    PEP_raw_cu    PEP_raw_zn  \
0   8.984628e+04           NaN  0.000000e+00           NaN           NaN   
1   7.444524e+04           NaN  0.000000e+00           NaN           NaN   
2            NaN           NaN  0.000000e+00           NaN           NaN   
3   0.000000e+00   

In [88]:
'''
NORMALIZE the EXISTING raw Pollutant Exceedance Potential scores held in pd_exPEP_raw to a new dataframe called pd_exPEP_norm.
Use calculation:
PEP_Norm = (PEP_raw - PEPmin) / (PEPMax - PEPmin)

Hold the PEPmax and PEPmin baseline scores used for the normalization in a dataframe called pd_NormBaselinePEP
****NOTE: LATER, we'll need to write the norm. basis to file
          This will allow us to use a common baseline in future (when we get more data, we'll want to have same baseline)
          
'''
###BUILD BASELINE dataframe pd_NormBaselinePEP:
##Write max and mins for each PEP_Raw constituent to a dict. Use 0 as min for all:
dict_NormBaselinePEP = {'PEP_Baseline_' + Constituent: [pd_exPEP_raw.loc[:,'PEP_raw_' + Constituent].max(),
                                             0]
                                             for Constituent in pollLS}
dict_NormBaselinePEP['MaxMin'] = ['Max','Min'] #add column identifying if row is max or min
pd_NormBaselinePEP = pd.DataFrame(dict_NormBaselinePEP) #write dict to new dataframe 
print ('This is the pd_NormBaselinePEP dataframe:')
display(pd_NormBaselinePEP)

#initialize pd_exPEP_norm dataframe w/ Facility_IDs from pd_exPEP_raw
pd_exPEP_norm = pd_exPEP_raw.loc[:,['Facility_ID']]
pd_exPEP_norm.reset_index(drop=True,inplace=True)

##calculate norm for each constituent:
def _HELPER_calc_PEP_norm(row, Constituent):
    #HELPER function to calculate norm of PEP_raw
    BLmax= pd_NormBaselinePEP.loc[pd_NormBaselinePEP['MaxMin']=='Max', 'PEP_Baseline_' + Constituent].values[0]
    BLmin= pd_NormBaselinePEP.loc[pd_NormBaselinePEP['MaxMin']=='Min', 'PEP_Baseline_' + Constituent].values[0]
    #i dont know how to catch div by zero errors using numpy :-/
    dv = row['PEP_raw_' + Constituent] - BLmin
    if dv != 0:
        v = (dv - BLmin)/(BLmax - BLmin)
    else:
        v = 0 #force to 0
    return v

#loop through each constituent; calculating norms as we go:   
for Constituent in pollLS:
        pd_exPEP_norm['PEP_norm_' + Constituent] = pd_exPEP_raw.apply(lambda row: 
                                           _HELPER_calc_PEP_norm(row,Constituent), axis = 1)
print('This is the pd_exPEP_norm dataframe:')
display(pd_exPEP_norm)

#TO DO:  WRITE NORMALIZED PEP SCORES TO DB: 

This is the pd_NormBaselinePEP dataframe:


MaxMin  PEP_Baseline_an  PEP_Baseline_cu  PEP_Baseline_fe  PEP_Baseline_n  \
0    Max     7.452262e+06     2.297250e+08     4.448060e+09    1.911565e+07   
1    Min     0.000000e+00     0.000000e+00     0.000000e+00    0.000000e+00   

   PEP_Baseline_nn  PEP_Baseline_og  PEP_Baseline_p  PEP_Baseline_phmax  \
0     2.600866e+06     3.126724e+07    7.283846e+06       497346.455645   
1     0.000000e+00     0.000000e+00    0.000000e+00            0.000000   

   PEP_Baseline_phmin  PEP_Baseline_tss  PEP_Baseline_turbidity  \
0       703512.890691      1.392570e+09            2.968967e+09   
1            0.000000      0.000000e+00            0.000000e+00   

   PEP_Baseline_zn  
0     1.051438e+09  
1     0.000000e+00

This is the pd_exPEP_norm dataframe:


Facility_ID  PEP_norm_tss  PEP_norm_turbidity  PEP_norm_p  PEP_norm_n  \
0             1      0.114158            0.020291    0.028180    0.207038   
1             2      0.001650            0.003014    0.001599    0.079701   
2             3           NaN                 NaN         NaN         NaN   
3             4      0.021782            0.000000    0.013846    0.006678   
4             5      0.062949            0.032630    0.116553    0.441888   
5             6      0.217220            0.216558    0.216123    0.215810   
6             7      0.000000            0.000300    0.000000    0.001177   
7             8           NaN                 NaN         NaN         NaN   
8             9           NaN            0.089554    0.089675    0.090066   
9            10      0.061154            0.060967    0.060845    0.060757   
10           11      0.523972            0.522376    0.521325    0.520570   
11           12      0.000996            0.000725    0.005346    0.012343   
12           13      0.018381            0.012025    0.017474    0.060604   
13           14      0.047870            0.047724    0.047628    0.047559   
14           15      0.044808            0.044671    0.044581    0.044517   
15           34      0.404911            0.403677    0.402865    0.402282   
16           41           NaN            0.926421    0.920292    0.920635   
17           81      0.000000            0.000000    0.000000    0.000000   
18           82           NaN                 NaN         NaN         NaN   
19           83           NaN            0.000546    0.002534    0.006131   
20           84      0.156674            0.156196    0.155882    0.155656   
21           85           NaN                 NaN         NaN         NaN   
22           92      0.000000            0.000000    0.000000    0.011762   
23           93           NaN            1.000000    1.000000    1.000000   
24           94           NaN                 NaN         NaN         NaN   
25           95           NaN            0.514231    0.514517    0.515927   
26           96           NaN            0.202733    0.202733    0.202733   
27           97      0.012880            0.004240    0.063989    0.024377   
28           98      1.000000            0.996954    0.994948    0.993507   
29           99      0.001739            0.000240    0.010137    0.031243   
30          100      0.000000            0.000000    0.006846    0.022851   
31          101           NaN                 NaN         NaN         NaN   
32          102           NaN            0.599137    0.599137    0.599137   
33          103      0.001224            0.003757    0.029766    0.161371   
34          104           NaN                 NaN         NaN         NaN   
35          105           NaN            0.165449    0.165449    0.165449   
36          106      0.011634            0.006149    0.104354    0.047742   
37          107      0.015367            0.010061    0.011674    0.017133   
38          108      0.970018            0.336729    0.327825    0.121071   
39          109           NaN                 NaN         NaN         NaN   
40          110           NaN            0.042434    0.445689    0.186190   
41          111           NaN                 NaN         NaN         NaN   
42          112           NaN            0.003658    0.083262    0.121372   
43          113      0.000000            0.000000    0.000000    0.000000   
44          114      0.000000            0.000000    0.000000    0.000000   

    PEP_norm_nn  PEP_norm_an  PEP_norm_og  PEP_norm_cu  PEP_norm_zn  \
0      0.034545          NaN     0.000000          NaN          NaN   
1      0.028623          NaN     0.000000          NaN          NaN   
2           NaN          NaN     0.000000          NaN          NaN   
3      0.000000          NaN     0.000000          NaN          NaN   
4      0.000844          NaN     0.022020          NaN          NaN   
5      0.207688          NaN     0.

In [95]:
'''
SUM NORMALIZED PEP SCORES
write scores to new dataframe called pd_PEP_sum
'''
def SumNormPEPs (pd_PEP_norm):
    #general function to sum normalized PEPs
    pd_PEP_norm.set_index('Facility_ID', inplace=True) #move FAcility ID to index temporarily
    pd_PEP_norm['PEP_norm_sum'] = pd_PEP_norm.sum(axis = 1) #sum norm scores for each facility
    pd_PEP_norm.reset_index(inplace=True) #move facility ID from index
    return(pd_PEP_norm.loc[:,['Facility_ID', 'PEP_norm_sum']]) #copy summed norm scores and return dataframe


pd_exPEP_sum = SumNormPEPs(pd_exPEP_norm)
display(pd_exPEP_sum)

Facility_ID  PEP_norm_sum
0             1      2.931968
1             2      0.343760
2             3      0.703905
3             4      0.126920
4             5      2.030654
5             6      4.523401
6             7      0.007883
7             8      0.118733
8             9      2.083914
9            10      1.273466
10           11     10.911221
11           12      0.130276
12           13      0.419768
13           14      0.996854
14           15      0.933076
15           34      8.431884
16           41     18.853209
17           81      0.000000
18           82      3.000000
19           83      0.202346
20           84      4.651508
21           85      0.000000
22           92      0.043652
23           93     21.428646
24           94      0.000000
25           95     11.960312
26           96      4.344286
27           97      0.316457
28           98     20.824052
29           99      0.130077
30          100      0.089091
31          101      2.093388
32          102     12.836759
33          103      0.753610
34          104      0.738081
35          105      3.545345
36          106      0.983393
37          107      0.166513
38          108     12.089478
39          109      5.374422
40          110      7.375548
41          111      1.155003
42          112      3.095303
43          113      0.000000
44          114      0.000000

In [111]:
'''
CALCULATE WRS PEP BASE SCORES & WRITE SCORES TO database (to be done later)
    WRS PEP BASE SCORE = NORM_PEP_SCORE*(SampleUncertainty + 1) 

'''
def _HELPER_PEPUncertainty(ls_id, dict_unc):
    '''determine the uncertainty level based on sample method
        (retrieve list of sample methods from ExPollConcs table for facilities in ls_id; assign uncertainty level using dict_unc)
       input: 
            ls_id: list of facility ids
            dict_unc: dictionary of uncertainty values for each sample method
        return: 
            pd_unc: dataframe [Facility_ID, UncertaintyValue]
    '''
    #get sample method for each facility in ls_id list
    q = session.query(ExPollConcs.facility_id.label('Facility_ID'), ExPollConcs.sample_method.label('sample_method')).filter(Facility_Chars.id.in_(ls_id)).distinct(ExPollConcs.facility_id).order_by(ExPollConcs.facility_id)
    pd_samplemethod = pd.read_sql(q.statement,session.bind)
    #use dict_unc to assign uncertainty value for each facility's sample method
    pd_samplemethod['Uncertainty_Value'] = pd_samplemethod['sample_method'].apply(lambda val: dict_unc[val])
    return pd_samplemethod
    
def CalcWRSPEPBaseScore(pd_PEP_sum):
    #calculate wrs pep base score = NORM_PEP_SCORE*(SampleUncertainty + 1) 
    #input: pd_PEP_sum dataframe containing COLUMNS [Facility_ID, PEP_norm_sum]
    #return: dataframe of WRS PEP Base Scores
    
    #make a list of Facility IDs in pd_PEP_sum
    ls_id = [np.asscalar(id) for id in pd_PEP_sum['Facility_ID']] #id given as numpy int. cast to python int https://stackoverflow.com/questions/9452775/converting-numpy-dtypes-to-native-python-types
    
    #make the pd_WRSPEPBaseScore dataframe:
    #write uncertainty information into pd_WRSPEPBaseScore
    pd_WRSPEPBaseScore = _HELPER_PEPUncertainty(ls_id, {'infield':0.25, 'sim_MaxType':1.0, 'sim_EMC':0.0})
    #copy in PEP_norm_sum values
    pd_WRSPEPBaseScore['PEP_norm_sum'] = pd_PEP_sum['PEP_norm_sum']
    #calculate PEP wrs and then write result into column
    pd_WRSPEPBaseScore['WRS_PEPBase'] = pd_WRSPEPBaseScore['PEP_norm_sum'] * (pd_WRSPEPBaseScore['Uncertainty_Value'] + 1)
    return pd_WRSPEPBaseScore
    
#calc WRS PEP Base Scores for existing normalized PEP sums (pd_exPEP_norm)
pd_exWRSPEPBaseScore = CalcWRSPEPBaseScore(pd_exPEP_sum)
display(pd_exWRSPEPBaseScore)


Facility_ID sample_method  Uncertainty_Value  PEP_norm_sum  WRS_PEPBase
0             1       infield               0.25      2.931968     3.664960
1             2       infield               0.25      0.343760     0.429700
2             3       infield               0.25      0.703905     0.879881
3             4       infield               0.25      0.126920     0.158650
4             5       infield               0.25      2.030654     2.538318
5             6   sim_MaxType               1.00      4.523401     9.046802
6             7       infield               0.25      0.007883     0.009854
7             8       infield               0.25      0.118733     0.148416
8             9   sim_MaxType               1.00      2.083914     4.167828
9            10   sim_MaxType               1.00      1.273466     2.546932
10           11   sim_MaxType               1.00     10.911221    21.822443
11           12       infield               0.25      0.130276     0.162845
12           13       infield               0.25      0.419768     0.524710
13           14   sim_MaxType               1.00      0.996854     1.993708
14           15   sim_MaxType               1.00      0.933076     1.866152
15           34   sim_MaxType               1.00      8.431884    16.863769
16           41   sim_MaxType               1.00     18.853209    37.706417
17           81       infield               0.25      0.000000     0.000000
18           82       infield               0.25      3.000000     3.750000
19           83       infield               0.25      0.202346     0.252932
20           84   sim_MaxType               1.00      4.651508     9.303017
21           85       infield               0.25      0.000000     0.000000
22           92       infield               0.25      0.043652     0.054565
23           93   sim_MaxType               1.00     21.428646    42.857293
24           94       infield               0.25      0.000000     0.000000
25           95   sim_MaxType               1.00     11.960312    23.920623
26           96   sim_MaxType               1.00      4.344286     8.688572
27           97       infield               0.25      0.316457     0.395571
28           98   sim_MaxType               1.00     20.824052    41.648105
29           99       infield               0.25      0.130077     0.162596
30          100       infield               0.25      0.089091     0.111364
31          101   sim_MaxType               1.00      2.093388     4.186775
32          102   sim_MaxType               1.00     12.836759    25.673517
33          103       infield               0.25      0.753610     0.942013
34          104   sim_MaxType               1.00      0.738081     1.476161
35          105   sim_MaxType               1.00      3.545345     7.090689
36          106       infield               0.25      0.983393     1.229241
37          107       infield               0.25      0.166513     0.208141
38          108       infield               0.25     12.089478    15.111847
39          109       infield               0.25      5.374422     6.718028
40          110       infield               0.25      7.375548     9.219435
41          111       infield               0.25      1.155003     1.443754
42          112       infield               0.25      3.095303     3.869129
43          113   sim_MaxType               1.00      0.000000     0.000000
44          114       infield               0.25      0.000000     0.000000
45          240       infield               0.25           NaN          NaN

In [15]:
'''


CALCULATION: WRS_pep = PEP_norm_sum * Category_RiskFactor

WE WILL NEED THIS LATER!

'''

#make riskfactor dataframe
#need to get risk factors from database for each facility in pd_exPEP_sum.
#facility risk factors are held in the Facility_Risks table. need to get existing_Facility_Risk_id field for each facility

# def GET_pd_facrisks(ls_id):
#     '''helper function that takes in list of facility_char ids and returns dataframe of:
#             facility_char.id
#             facility_char.existing_facility_risk_id
#             Facility_Risks.category_risk_factor
#             Facility_Risks.wrs_pollutant_base_risks_id
#     '''    
#     q_facriskIDs =  session.query(Facility_Chars.existing_facility_risk_id).filter(Facility_Chars.id.in_(ls_id)) #for facilities in pd_exPEP_sum, get existing_facility_risk_id records
#     #use q_facriskIDs as filter on Facility_Risks table to get associated wrs pollutant base id
#     q_facrisks = session.query(
#         Facility_Chars.id.label('Facility_ID'), Facility_Chars.existing_facility_risk_id, Facility_Risks.wrs_pollutant_base_risks_id, Facility_Risks.Category_RiskFactor
#         ).filter(
#             Facility_Risks.id.in_(q_facriskIDs)).filter(
#                 Facility_Risks.id == Facility_Chars.existing_facility_risk_id).order_by(Facility_Chars.id)
#     pd_facrisks = pd.read_sql(q_facrisks.statement, session.bind)
#     return pd_facrisks

# def Calc_WRS_PEP(pd_PEP_sum):
#     #calculate the wrs pep sums for each facility 
#     #This is a list of facilities contained in pd_exPEP_sum (we'll need this list later)
#     ls_id = [np.asscalar(id) for id in pd_exPEP_sum['Facility_ID']] #id given as numpy int. cast to python int https://stackoverflow.com/questions/9452775/converting-numpy-dtypes-to-native-python-types
#     pd_facrisks = GET_pd_facrisks(ls_id) #get facility risk related data for each facility
#     #make dataframe using PEP_norm_sums and Category_RiskFactors for each facility (we can do this b/c the 2 dataframe share common facility id order)
#     pd_PEP_WRS = pd_exPEP_sum.loc[:,['Facility_ID']]
#     pd_PEP_WRS.reset_index(drop=True,inplace=True)
#     pd_PEP_WRS['WRS_PEP'] = pd_exPEP_sum['PEP_norm_sum'] * pd_facrisks['Category_RiskFactor']
# display(pd_PEP_WRS)

# #TO DO LATER: WRITE IN TOTAL SCORES TO DB



'\n\n\nCALCULATION: WRS_pep = PEP_norm_sum * Category_RiskFactor\n\nWE WILL NEED THIS LATER!\n\n'

In [33]:
# %%capture cap --no-stderr
# EVALUATE BASE BMP FEASIBILITY at each facility
# Write results to the base_bmp_feasibility_test_results table.

print('\n******Evaluating Base BMP feasibility at facilities.******')
ShowCalculations = False #flag indicating if steps should be outputted
Expr.ResetEvalErrorCount() #RESET EXPRESION EVALUATOR ERROR COUNT

#Only analyze bmps at facilities we have data for. make list of these facilities.
ls_id = [np.asscalar(id) for id in pd_exPEP_sum['Facility_ID']] #id given as numpy int. cast to python int https://stackoverflow.com/questions/9452775/converting-numpy-dtypes-to-native-python-types
for aFac in session.query(Facility_Chars).filter(Facility_Chars.id.in_(ls_id)):    
    if ShowCalculations: print ('\n***Evaluating base bmp feasibiilty tests for facility: ', aFac.Fac_Name), ' ***'
    myBMPs = session.query(Base_BMPs)
    for aBMP in myBMPs:
        if ShowCalculations:print ('\n######Evaluating feasibility of base_bmp: ', aBMP.bmp_name, ' ID: ', aBMP.id, '######')
        BBMP_Eval.Eval_base_bmp_feasibility_tests(aFac, aBMP, ShowCalculations)
session.commit
winsound.Beep(250,1000)
print ('*****************************************************************')
print ('* Completed evaluating Base BMP feasibility                     *')
if Expr.CountEvalErrors() >0:
    print (Expr.CountEvalErrors(), ' errors were encountered. Review output to identify location(s)')
    print ('Hint: expression evaluation error lines are prefixed by: FAULT!!!! Error occured while evaluating expression:')
else:
    print ('No errors detected.')
print ('*****************************************************************')

# with open('output.txt', 'w') as f:
#     f.write(cap.stdout)
# f.close()


******Evaluating Base BMP feasibility at facilities.******
*****************************************************************
* Completed evaluating Base BMP feasibility                     *
54  errors were encountered. Review output to identify location(s)
Hint: expression evaluation error lines are prefixed by: FAULT!!!! Error occured while evaluating expression:
*****************************************************************


In [16]:
#CREATE COMBO BMPS USING BASE BMPS
#ALL POSSIBLE COMBOS WILL BE CREATED AND ADDED TO THE COMBO_BMPS TABLE
#MAXIMUM POLLUTANT REMOVAL RATES ARE DETERMINED BY IDENTIFYING 
#  THE BASE_BMP IN THE COMBO THAT PROVIDES THE HIGHEST REMOVAL RATE FOR A GIVEN POLLUTANT

import time
print ('get a coffee...this one takes a while!')
start_time = time.time()
CBMP_Eval.Make_ALL_bmp_base_option_combos()
session.commit()
print ('--- %s execution time in seconds ---' % (time.time() - start_time))

get a coffee...this one takes a while!
 Making BMP Combos of length: 1
 Find max pollutant removal rates for each BMP Combo of length:  1
  Made  13  combos
 Making BMP Combos of length: 2
 Find max pollutant removal rates for each BMP Combo of length:  2
  Made  78  combos
 Making BMP Combos of length: 3
 Find max pollutant removal rates for each BMP Combo of length:  3
  Made  286  combos
 Making BMP Combos of length: 4
 Find max pollutant removal rates for each BMP Combo of length:  4
  Made  715  combos
 Making BMP Combos of length: 5
 Find max pollutant removal rates for each BMP Combo of length:  5
  Made  1287  combos
 Making BMP Combos of length: 6
 Find max pollutant removal rates for each BMP Combo of length:  6
  Made  1716  combos
 Making BMP Combos of length: 7
 Find max pollutant removal rates for each BMP Combo of length:  7
  Made  1716  combos
 Making BMP Combos of length: 8
 Find max pollutant removal rates for each BMP Combo of length:  8
  Made  1287  combos
 Making

In [56]:
'''
Identify the feasible bmp combinations for each facility
Use base bmp feasibility results for each facility.
Put results into the combo_bmp_feasibility_test_results table
'''
import itertools     #https://docs.python.org/3/library/itertools.html    
import pandas as pd

from sqlalchemy import and_

def _Make_bmp_fingerprint(base_BMP_components):
    #create fingerprint of the passed list of base_bmp_ids
    #fingerprint is just a | separated list of ids of the base bmps that make up the combo bmp
    #corresponds to bmp_options table's bmp_fingerprint field
    #FORMAT: |bmp_option_base_component_id||bmp_option_base_component_id| w/ id's given in ascending order
    fingerprint = '|' + '|'.join(str(id) + '|' for id in base_BMP_components)
    return fingerprint

def Eval_FacBMPCombo(pd_basebmps, myFacility, bmpCombo):
    '''
    input:
        pdbasebmps: pandas built from a BBMP_Eval.evalFacility_BaseBMP dictionary list
                    assme that pandas is passed in w/ index is set as base_bmp_id
        myFacility: SQLA fac_chars record
        bmpCombo: list of base_bmp_ids that make up this combo
    
    #retrieve previously computed combo removal rate
    #calculate combo cip and om cost, insert/update database
    #calculate wrs reduction, insert/update database

    '''    
    #get combo bmp pollutant removal rates into pandas 
    q = session.query(Combo_BMPs.bmp_fingerprint, Combo_BMPs.id.label('combos_bmp_id'), PRR.id.label('PRR_id'),
          PRR.r_tss, PRR.r_turbidity, PRR.r_p, PRR.r_n, PRR.r_nn, PRR.r_an,
          PRR.r_og, PRR.r_cu, PRR.r_zn, PRR.r_fe, PRR.r_phmin, PRR.r_phmax
        ).filter(Combo_BMPs.bmp_fingerprint == _Make_bmp_fingerprint(bmpCombo)).filter(
        Combo_BMPs.bmp_option_removal_rate_id == PRR.id)  
    pd_rr = pd.read_sql(q.statement,session.bind) 

    #use information in pd_rr to get CBFTR_record - make new record if necessary
    myCBFTR = Base.metadata.tables['combo_bmp_feasibility_test_results']
    myCBFTR_id = SQLA_main.insertupdateRec(myCBFTR,{'facility_id':myFacility.id, 'combo_bmps_id':pd_rr['combos_bmp_id'][0]},
                and_(
        myCBFTR.c['facility_id'] == myFacility.id,
        myCBFTR.c['combo_bmps_id'] == pd_rr['combos_bmp_id'][0]
                    ))
    session.flush()
  
    #get costs in pandas
    sumCIP = sum(pd_basebmps.loc[bmp_id,'calc_cip_cost'] for bmp_id in bmpCombo)
    sumOM = sum(pd_basebmps.loc[bmp_id,'calc_om_cost'] for bmp_id in bmpCombo)
    pd_sums = pd.DataFrame([{'calc_cip_cost':sumCIP, 'calc_om_cost': sumOM}])

    #prepare dataframe to return:
    pd_temp = pd.concat([pd_rr, pd_sums], axis = 1)#make one dataframe containing combo bmp's removal rates and costs:
    pd_temp.insert(0,'Facility_ID',myFacility.id) #include facility id column
    return (pd_temp)
    

def Eval_FacBMPCombos(aFac, ShowCalculations):
    #a wrapper around Eval_FacBMPCombo
    print('\n***Evaluating feasible bmp combos for facility: ', aFac.Fac_Name, '***')
    print ('****Evaluating feasibile base bmps****')
    df = pd.DataFrame(BBMP_Eval.evalFacility_BaseBMP(aFac, False)).set_index('base_bmp_id')
    if ShowCalculations: display (df)   
    df = df.loc[df['is_feasible'] == 1]
    if ShowCalculations:
        print ('****These are the feasible base bmps. I\'ll use them to make combos:****')
        display (df)
    feas_ls = df.index #send feasible base bmp ids to list
    pd_temp = pd.DataFrame()
    for CBOLen in range (1, len(feas_ls)+1): #+1 so it's inclusive of last count
        for combo in  itertools.combinations(feas_ls,CBOLen):
            pd_temp = pd.concat([pd_temp,Eval_FacBMPCombo(df,aFac, list(combo))])
#             print ('Here is a summary of the combo: ', list(combo))
#             display(Eval_FacBMPCombo(df,aFac, list(combo)))
    print ('      There are ', len(pd_temp), ' combinations of feasible Base BMPs.')
    return pd_temp

            
def Eval_All_FacBMPCombos():
    ShowCalculations = False
    print ('Evaluating feasibile BMP Options for each facility:')
    #Only analyze bmps at facilities we have data for. make list of these facilities.
    ls_id = [np.asscalar(id) for id in pd_exPEP_sum['Facility_ID']] #id given as numpy int. cast to python int https://stackoverflow.com/questions/9452775/converting-numpy-dtypes-to-native-python-types
    pd_BaseBMPCombos = pd.DataFrame() #create empty dataframe that we'll fill w/ the feasibile combos
    for aFac in session.query(Facility_Chars).filter(Facility_Chars.id.in_(ls_id)):        
#         Eval_FacBMPOptions(aFac, ShowCalculations)
        pd_BaseBMPCombos = pd.concat([pd_BaseBMPCombos, Eval_FacBMPCombos(aFac, ShowCalculations)])
    return pd_BaseBMPCombos

pd_BaseBMPCombos = Eval_All_FacBMPCombos()
display(pd_BaseBMPCombos)
session.commit()

Evaluating feasibile BMP Options for each facility:

***Evaluating feasible bmp combos for facility:  Kalihi-Palama Bus & Paratransit Facility ***
****Evaluating feasibile base bmps****
      There are  15  combinations of feasible Base BMPs.

***Evaluating feasible bmp combos for facility:  Pearl City Bus Facility ***
****Evaluating feasibile base bmps****
      There are  15  combinations of feasible Base BMPs.

***Evaluating feasible bmp combos for facility:  Kapaa Refuse Transfer Station ***
****Evaluating feasibile base bmps****
      There are  15  combinations of feasible Base BMPs.

***Evaluating feasible bmp combos for facility:  Kawailoa Refuse Transfer Station ***
****Evaluating feasibile base bmps****
      There are  3  combinations of feasible Base BMPs.

***Evaluating feasible bmp combos for facility:  Keehi Refuse Transfer Station ***
****Evaluating feasibile base bmps****
      There are  31  combinations of feasible Base BMPs.

***Evaluating feasible bmp combos for fa

Facility_ID   bmp_fingerprint  combos_bmp_id  PRR_id  r_tss  r_turbidity  \
0             1               |4|              4      17   0.90          0.0   
0             1               |6|              6      19   0.80          0.0   
0             1               |8|              8      21   0.95          0.0   
0             1               |9|              9      22   1.00          1.0   
0             1            |4||6|             48      61   0.90          0.0   
0             1            |4||8|             50      63   0.95          0.0   
0             1            |4||9|             51      64   1.00          1.0   
0             1            |6||8|             65      78   0.95          0.0   
0             1            |6||9|             66      79   1.00          1.0   
0             1            |8||9|             77      90   1.00          1.0   
0             1         |4||6||8|            267     280   0.95          0.0   
0             1         |4||6||9|            268     281   1.00          1.0   
0             1         |4||8||9|            279     292   1.00          1.0   
0             1         |6||8||9|            328     341   1.00          1.0   
0             1      |4||6||8||9|            917     930   1.00          1.0   
0             2               |4|              4      17   0.90          0.0   
0             2               |6|              6      19   0.80          0.0   
0             2               |8|              8      21   0.95          0.0   
0             2               |9|              9      22   1.00          1.0   
0             2            |4||6|             48      61   0.90          0.0   
0             2            |4||8|             50      63   0.95          0.0   
0             2            |4||9|             51      64   1.00          1.0   
0             2            |6||8|             65      78   0.95          0.0   
0             2            |6||9|             66      79   1.00          1.0   
0             2            |8||9|             77      90   1.00          1.0   
0             2         |4||6||8|            267     280   0.95          0.0   
0             2         |4||6||9|            268     281   1.00          1.0   
0             2         |4||8||9|            279     292   1.00          1.0   
0             2         |6||8||9|            328     341   1.00          1.0   
0             2      |4||6||8||9|            917     930   1.00          1.0   
..          ...               ...            ...     ...    ...          ...   
0           112               |6|              6      19   0.80          0.0   
0           112               |8|              8      21   0.95          0.0   
0           112               |9|              9      22   1.00          1.0   
0           112              |10|             10      23   0.12          0.0   
0           112            |4||6|             48      61   0.90          0.0   
0           112            |4||8|             50      63   0.95          0.0   
0           112            |4||9|             51      64   1.00          1.0   
0           112           |4||10|             52      65   0.90          0.0   
0           112            |6||8|             65      78   0.95          0.0   
0           112            |6||9|             66      79   1.00          1.0   
0           112           |6||10|             67      80   0.80          0.0   
0           112            |8||9|             77      90   1.00          1.0   
0           112           |8||10|             78      91   0.95          0.0   
0           112           |9||10|             82      95   1.00          1.0   
0           112         |4||6||8|            267     280   0.95          0.0   
0           112         |4||6||9|            268     281   1.00          1.0   
0           112        |4||6||10|            269     282   0.90          0.0   
0           112         |4||8||9|            279     292   1.00          1.0   
0   

In [18]:
# session.close()
# engine.dispose()

In [19]:
# http://pythonhow.com/accessing-dataframe-columns-rows-and-cells/
import pandas as pd #import in pandas library
print ('#get csv data and read into pandas')
df1=pd.read_csv("http://pythonhow.com/wp-content/uploads/2016/01/Income_data.csv")
print (df1)
print ('#write new dataframe w/ index set to the "State" column in the csv')
df2=df1.set_index("State").copy()
print (df2)
print ('#extract a portion of the dataframe: States = Alaska to Arkansas; and Dates 2005:2007')
print (df2.loc["Alaska":"Arkansas","2005":"2007"])

print ('Get only certain States, using a list of states:')
getStates = ['Alaska', 'Arizona']
print (df2.loc[getStates])

print ('#slice a column:')
df2.loc[: , "2005"]
print ('get a cell:')
df2.loc['Alaska','2005']
print ('#get max of 2005 data')
print (df2.loc[:,'2005'].max())
print ('take 2005 column and put into list')
LS = df2['2005'].tolist() #this is a series. we use the .tolist() to convert from series to list
print (type(LS))


#get csv data and read into pandas
       GEOID       State   2005   2006   2007   2008   2009   2010   2011  \
0  04000US01     Alabama  37150  37952  42212  44476  39980  40933  42590   
1  04000US02      Alaska  55891  56418  62993  63989  61604  57848  57431   
2  04000US04     Arizona  45245  46657  47215  46914  45739  46896  48621   
3  04000US05    Arkansas  36658  37057  40795  39586  36538  38587  41302   
4  04000US06  California  51755  55319  55734  57014  56134  54283  53367   

    2012   2013  
0  43464  41381  
1  63648  61137  
2  47044  50602  
3  39018  39919  
4  57020  57528  
#write new dataframe w/ index set to the "State" column in the csv
                GEOID   2005   2006   2007   2008   2009   2010   2011   2012  \
State                                                                           
Alabama     04000US01  37150  37952  42212  44476  39980  40933  42590  43464   
Alaska      04000US02  55891  56418  62993  63989  61604  57848  57431  63648   
Ari

In [20]:
df = pd.DataFrame({'col1' : [1.0] * 5, 
                   'col2' : [2.0] * 5, 
                   'col3' : [3.0] * 5 }, index = range(1,6),)
display(df)
df2 = pd.DataFrame({'col1' : [10.0] * 5, 
                    'col2' : [100.0] * 5, 
                    'col3' : [1000.0] * 5 }, index = range(1,6),)
display(df2)
df.mul(df2, 0) # element by element multiplication no problems

col1  col2  col3
1   1.0   2.0   3.0
2   1.0   2.0   3.0
3   1.0   2.0   3.0
4   1.0   2.0   3.0
5   1.0   2.0   3.0

col1   col2    col3
1  10.0  100.0  1000.0
2  10.0  100.0  1000.0
3  10.0  100.0  1000.0
4  10.0  100.0  1000.0
5  10.0  100.0  1000.0

col1   col2    col3
1  10.0  200.0  3000.0
2  10.0  200.0  3000.0
3  10.0  200.0  3000.0
4  10.0  200.0  3000.0
5  10.0  200.0  3000.0

In [21]:
import datetime

# xmin = datetime.datetime.strptime('1/1/2018', "%m/%d/%Y").date()
# xmax = datetime.datetime.strptime('5/6/2018', "%m/%d/%Y").date()

# xmin <= datetime.date(2018,1,5) <= xmax

#     Wet Season is from: January 1 through April 30 and November 1 through December 31
#     Dry Season is from: May 1 through October 31

SampleDate = datetime.date(2018,11,1)

#Wet Season 1:
if datetime.date(SampleDate.year, 1,1) <= SampleDate <= datetime.date(SampleDate.year, 4,30):
    print ('ws 1')
elif datetime.date(SampleDate.year, 5,1) <= SampleDate <= datetime.date(SampleDate.year, 10,31):
    print ('dry')
else:
    print ('ws 2')
    
    
import numpy as np    
# np.max([float('nan'),0])
np.max([0,float('nan')])

if math.isnan(10)

SyntaxError: invalid syntax (<ipython-input-21-9e298a6b06aa>, line 26)

In [ ]:
#import the pandas library
import pandas as pd

ipl_data = {'Team': ['Riders', 'Riders', 'Devils', 'Devils', 'Kings',
         'kings', 'Kings', 'Kings', 'Riders', 'Royals', 'Royals', 'Riders'],
         'Rank': [1, 2, 2, 3, 3,4 ,1 ,1,2 , 4,1,2],
         'Year': [2014,2015,2014,2015,2014,2015,2016,2017,2016,2014,2015,2017],
         'Points':[876,789,863,673,741,812,756,788,694,701,804,690]}
df = pd.DataFrame(ipl_data)

display (df)

display (df.groupby('Team').groups)


import numpy as np


grouped = df.groupby('Year')
print (grouped['Points'].agg(np.max))

In [ ]:
df = pd.DataFrame({'col1': [1, 2], 'col2': [0.1, 0.2]},
                      index=['a', 'b'])
for idx in df.index:
    print (idx)
    print (df.loc[idx]['col2'])

In [ ]:
import pandas as pd
import numpy as np
data = {'name': ['Jason', 'Molly', 'Tina', 'Jake', 'Amy'], 
        'year': [2012, 2012, 2013, 2014, 2014], 
        'reports': [4, 24, 31, 2, 3],
        'coverage': [25, 94, 57, 62, 70]}
df = pd.DataFrame(data, index = ['Cochice', 'Pima', 'Santa Cruz', 'Maricopa', 'Yuma'])
display(df)

dfx = df.copy()

capitalizer = lambda x: x.upper()

dfx['name'] = dfx['name'].apply(capitalizer)
display(df)
display(dfx)

In [ ]:
def div0( a, b ):
    """ ignore / 0, div0( [-1, 0, 1], 0 ) -> [0, 0, 0] """
    with np.errstate(divide='ignore', invalid='ignore'):
        c = np.true_divide( a, b )
        c[ ~ np.isfinite( c )] = 0  # -inf inf NaN
    return c

div0( [-1, 0, 1], 0 )


In [ ]:
afile = open("testfile.txt","a") 
# file.write("Hello World") 
# file.write("This is our new text file") 
# file.write("and this is another line.") 
# file.write("Why? Because we can.") 
print ("test", file=afile)
afile.close() 